In [1]:
import requests
from bs4 import BeautifulSoup


# male eng name
response = requests.get("https://blog.amazingtalker.com/zh-hk/hk-eng/%e3%80%90%e8%8b%b1%e6%96%87%e5%90%8d%e5%ad%97%e3%80%91400-%e5%80%8b%e7%94%b7%e7%94%9f%e8%8b%b1%e6%96%87%e5%90%8d%e5%ad%97%e6%84%8f%e7%be%a9%e7%b8%bd%e6%95%b4%e7%90%86%ef%bc%8c%e6%89%be%e8%8b%b1/5169/")
soup = BeautifulSoup(response.text, "html.parser")
sel = soup.select("td")

male_name = []
for i in range(len(sel)):
    if i%4 == 0: male_name.append(sel[i].text)
# print(male_name)

# female eng name
response = requests.get("https://blog.amazingtalker.com/zh-tw/zh-eng/%e5%a5%b3%e7%94%9f%e8%8b%b1%e6%96%87%e5%90%8d%e5%ad%97%e6%84%8f%e7%be%a9%e7%b8%bd%e6%95%b4%e7%90%86/3696/")
soup = BeautifulSoup(response.text, "html.parser")
sel = soup.select("td")

female_name = []
for i in range(len(sel)):
    if i%4 == 0: female_name.append(sel[i].text)
# print(female_name)

# family name
response = requests.get("https://www.ef.com/wwen/english-resources/english-names/")
soup = BeautifulSoup(response.text, "html.parser")
# print(soup.prettify())
sel = soup.select("td")
# print(sel)

fami_name = []
for i in range(100, len(sel)):
    if i%4 != 0: 
        fami_name.append(sel[i].text)
# print(fami_name)

## Data Schema
# 
Customer(_**CID**_, password, Firstname, Lastname, gender, BDate, City)
# 
Cart(_**CartID**_, _CID_, _PID_, cart_status)
# 
Product (_**PID**_, PName, Size, Price, Category)
# 
Order(_**orderID**_, _CartID_, QTY, order_status, order_date, transaction_price)
# 
Warehouse(_**WID**_, location)
# 
Supplier(_**SID**_, SName)
# 
Store(_**PID**, _**WID**_)
# 
Supply(_**PID**, _**SID**_)

In [3]:
import csv
import random
import secrets
import time

def random_date(start, end, prop):
    format = '%Y-%m-%d'
    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))
    ptime = stime + prop * (etime - stime)
    return time.strftime(format, time.localtime(ptime))

In [3]:
# generate Customer(CID, Firstname, Lastname, gender, BDate)

PSWRD_LEN = 6
max_CID = 1 # intialize Customer ID from 0
N = 100 # generate N tuples
female_proportion = 0.6 # assume 60% of customers are female
city_list = ['Taipei', 'New Taipei', 'Hsinchu', 'Taichung', 'Tainan', 'Kaoshiung']

with open('Customer.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['CID', 'password', 'Firstname', 'Lastname', 'gender', 'BDate', 'City'])
    for i in range(N):
        Lastname = random.choice(fami_name)
        City = random.choice(city_list)
        pswrd = secrets.token_urlsafe(PSWRD_LEN)
        # generate gender and Fname
        g = random.random()
        if g < female_proportion:
            Fname = random.choice(female_name)
            gender = 'female'
        else:
            Fname = random.choice(male_name)
            gender = 'male'
        # generate BDate
        BDate = random_date("1971-1-1", "2010-1-1", random.random())
        writer.writerow([max_CID, pswrd, Fname, Lastname, gender, BDate, City])
        max_CID += 1
        

In [4]:
# generate Product(PID, PName, Size, Price, Category)

N = 100 # generate N tuples
max_PID = 1 # intialize Cart ID from 0
SIZE_LIST = ['XS', 'S', 'M', 'L', 'XL']
product_category = ['sweater', 'shoe', 'shirts', 'pants', 'skirts', 'T-shirt']
product_name = ['']

with open('Product.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['PID', 'PName', 'Size', 'Price', 'Category'])
    for i in range(N):
        PName = random.choice(product_name)
        Size = random.choice(SIZE_LIST)
        price = random.randint(100, 1000)
        Category = random.choice(product_category)
        writer.writerow([max_PID, PName, Size, price, Category])
        max_PID += 1

In [5]:
# generate Cart(CartID, CID, PID, cart_status)
# and Order(orderID, CartID, QTY, order_status, order_date, transaction_price)

N = 1000 # generate N tuples
CART_STAT = ['in_cart', 'ordered', 'deleted']
ORDER_STAT = ['ordered', 'in shipping', 'finished']
max_CartID = 1 # intialize Cart ID from 0
MAX_QANT = 10
max_orderID = 1 # intialize order ID from 0

with open('Cart.csv', 'w', newline='') as cart_csvfile:
    with open('Order.csv', 'w', newline='') as order_csvfile:
        cart_writer = csv.writer(cart_csvfile)
        order_writer = csv.writer(order_csvfile)
        cart_writer.writerow(['CartID', 'CID', 'PID', 'cart_status'])
        order_writer.writerow(['orderID', 'CartID', 'QTY', 'order_status', 'order_date', 'transaction_price'])
        for i in range(N):
            CID = random.randint(1, max_CID-1)
            PID = random.randint(1, max_PID-1)
            cart_status = random.choice(CART_STAT)
            cart_writer.writerow([max_CartID, CID, PID, cart_status])
            if cart_status == 'ordered':
                quantity = random.randint(1, MAX_QANT)
                order_date = random_date("2019-1-1", "2020-12-31", random.random())
                order_status = random.choice(ORDER_STAT)
                unit_price = 100 # unit price haven't been set
                transaction_price = quantity * unit_price
                order_writer.writerow([max_orderID, max_CartID, quantity, order_status, order_date, transaction_price])
                max_orderID += 1
            max_CartID += 1

In [6]:
# generate Warehouse(WID, location)
N = 10 # generate N tuples
max_WID = 1 # intialize Warehouse ID from 0
warehouse_location = ['Taipei', 'Taichung', 'Kaoshiung']

with open('Warehouse.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['WID', 'location'])
    for i in range(N):
        location = random.choice(warehouse_location)
        writer.writerow([max_WID, location])
        max_WID += 1

# generate Supplier(SID, SName)
N = 5 # generate N tuples
max_SID = 1 # intialize Supplier ID from 0
supplier_name = ['Zara', 'Adidas', 'Nike', 'NET', 'H&M']

with open('Supplier.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['SID', 'SName'])
    for i in range(N):
        writer.writerow([max_SID, supplier_name[i]])
        max_SID += 1

In [5]:
import csv
import random
import secrets

max_PID = 101
max_SID = 6
max_WID = 11

In [6]:
# Store(PID, WID)
with open('Store.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['PID', 'WID', 'quantity'])
    for i in range(1, max_PID):
        w_num = random.randint(1, max_WID-1)
        w_sample = random.sample(range(1, max_WID), w_num)
        for w in w_sample:
            q = random.randint(100, 500)
            writer.writerow([i, w, q])

# Supply(PID, SID)
with open('Supply.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['PID', 'SID', 'unit_cost'])
    for i in range(1, max_PID):
        s_num = random.randint(1, max_SID-1)
        s_sample = random.sample(range(1, max_SID), s_num)
        unit_cost_base = random.randint(50, 300)
        sold_price = int(unit_cost_base * random.uniform(1.5, 3))
        for s in s_sample:
            unit_cost = int(unit_cost_base * random.uniform(1, 1.3))
            writer.writerow([i, s, unit_cost, sold_price])